In [1]:
%matplotlib widget

from pathlib import Path
import sys

# table_analysis.ipynb is in script/analysis → project root is two levels up
project_root = Path.cwd().parents[1]
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import os
from datetime import datetime, timedelta
from torch_geometric.utils import from_scipy_sparse_matrix
from script import utility


In [2]:
def partition_nodes_to_cloudlets_by_range_proximity(cloudlets, radius_km, dataset_name):
    data_dir = project_root / "data" / dataset_name
    csv_path = data_dir / "locations.csv"
    locations_data = pd.read_csv(str(csv_path))

    cloudlet_nodes_list = [[] for _ in range(len(cloudlets))]

    for idx, sensor in locations_data.iterrows():
        sensor_loc = (sensor['latitude'], sensor['longitude'])
        closest_cloudlet = None
        min_distance = float('inf')

        for name, loc in cloudlets.items():
            if utility.is_within_radius(sensor_loc[0], sensor_loc[1], loc['lat'], loc['lon'], radius_km):
                distance = utility.calculate_distance(sensor_loc[0], sensor_loc[1], loc['lat'], loc['lon'])
                if distance < min_distance:
                    min_distance = distance
                    closest_cloudlet = loc['id']

        if closest_cloudlet is not None:
            cloudlet_nodes_list[closest_cloudlet].append(idx)

    return cloudlet_nodes_list

In [3]:
# Get num of nodes for each cloudlet (PeMS-BAY dataset)
def get_num_nodes_per_cloudlet(dataset, cloudlet_location_data):
    base_dir = project_root
    cloudlet_info_path = base_dir / "locations" / dataset / "locations.json"

    cloudlet_data_json = utility.load_json_file(cloudlet_info_path)
    cloudlets, radius_km = utility.get_cloudlet_location_info_from_json(cloudlet_location_data, cloudlet_data_json)

    cln_nodes_list = partition_nodes_to_cloudlets_by_range_proximity(cloudlets, radius_km, dataset)

    return cln_nodes_list

In [4]:
def compute_weighted_final_metrics(experiment_name: str, cloudlet_sizes: dict):
     exp_dir = project_root / "logs" / experiment_name
     val_metric_dir = exp_dir / "val_metric"
     new_val_metric_dir = exp_dir / "new_val_metric"

     w_sum_mae = 0.0; w_sum_rmse = 0.0; w_sum_wmape = 0.0; w_sum_sudden_rate = 0.0
     total_w = 0

      # helper: robust column getter (exact name or close variants)
     def get_col(df, candidates):
        for c in candidates:
            if c in df.columns:
                return df[c]
        # try a more tolerant match
        norm = {col.lower().replace("  ", " ").strip(): col for col in df.columns}
        for c in candidates:
            key = c.lower().replace("  ", " ").strip()
            if key in norm:
                return df[norm[key]]
        raise KeyError(f"None of the columns found: {candidates} in {list(df.columns)}")
     
     for cln_id, size in cloudlet_sizes.items():
        vm_path = val_metric_dir / f"masked_{cln_id}.csv"
        nvm_path = new_val_metric_dir / f"masked_{cln_id}.csv"

        if not vm_path.exists():
            print(f"Warning: missing {vm_path}, skipping cloudlet {cln_id}")
            continue
        if not nvm_path.exists():
            print(f"Warning: missing {nvm_path}, skipping cloudlet {cln_id}")
            continue
        
        try:
            vm = pd.read_csv(vm_path)
            nvm = pd.read_csv(nvm_path)

            # Use the last non-NaN row as "final epoch"
            vm_last = vm.dropna(how="any").tail(1)
            nvm_last = nvm.dropna(how="any").tail(1)
            if vm_last.empty or nvm_last.empty:
                print(f"Warning: empty final row in cloudlet {cln_id}, skipping")
                continue

            mae = float(get_col(vm_last, ["MAE"]).iloc[0])
            rmse = float(get_col(vm_last, ["RMSE"]).iloc[0])
            wmape = float(get_col(vm_last, ["WMAPE"]).iloc[0])

            sudden_rate = float(get_col(
                nvm_last,
                [
                    "Total sudden change in speed rate",
                    # fallback aliases just in case
                    "Total sudden change in speed rate (%)",
                    "Sudden change rate",
                ],
            ).iloc[0])

            # accumulate weighted sums
            w_sum_mae += mae * size
            w_sum_rmse += rmse * size
            w_sum_wmape += wmape * size
            w_sum_sudden_rate += sudden_rate * size
            total_w += size
            
        except Exception as e:
            print(f"Error processing cloudlet {cln_id}: {e}")
            continue
     if total_w == 0:
        raise RuntimeError("No valid cloudlet metrics found to aggregate.")
     
     weighted = {
        "MAE": w_sum_mae / total_w,
        "RMSE": w_sum_rmse / total_w,
        "WMAPE": w_sum_wmape / total_w,  # keep as fraction if your CSV is fraction; multiply by 100 if you want %
        "Total sudden change in speed rate": w_sum_sudden_rate / total_w,  # same note on %
    }
     
     return weighted

In [28]:
cln_nodes_list = get_num_nodes_per_cloudlet("pems-bay", "experiment_1")

cloudlet_sizes = {i: len(nodes) for i, nodes in enumerate(cln_nodes_list)}
print(f"cloudlet_sizes: {cloudlet_sizes}")
# Result:
# cln_0 - 37
# cln_1 - 27
# cln_2 - 33
# cln_3 - 59
# cln_4 - 75
# cln_5 - 51
# cln_6 - 43

/home/ubuntu/kralj/stgcn-distributed/locations/pems-bay/locations.json
cloudlet_nodes_list: [[22, 27, 51, 61, 75, 76, 80, 103, 119, 146, 156, 157, 178, 179, 180, 183, 188, 189, 223, 224, 225, 247, 249, 287, 288, 289, 291, 292, 293, 297, 299, 300, 301, 302, 303, 305, 309], [11, 29, 32, 70, 71, 77, 85, 91, 92, 113, 114, 172, 173, 194, 195, 239, 252, 286, 290, 294, 295, 296, 298, 304, 306, 307, 308], [9, 14, 17, 25, 30, 31, 40, 45, 49, 54, 55, 64, 79, 84, 93, 107, 109, 121, 122, 133, 135, 138, 186, 187, 218, 219, 220, 221, 222, 246, 248, 250, 251], [0, 2, 4, 13, 19, 23, 26, 36, 38, 56, 68, 82, 105, 118, 120, 124, 126, 131, 134, 153, 154, 158, 170, 171, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 216, 217, 253, 254, 258, 259, 260, 261, 262, 264, 268, 274, 275, 276, 278, 279, 281, 315, 316, 317, 318, 319, 320], [6, 8, 18, 20, 24, 37, 41, 53, 57, 63, 72, 73, 78, 89, 90, 97, 102, 110, 111, 125, 130, 132, 137, 148, 150, 151, 160, 165, 166, 167, 168, 169, 174, 175, 176, 177, 181

In [ ]:
# PeMS-BAY, 70dps, 15min, trad-fl, full graph connectivity
experiment = "2025-09-05_20-33-32_pems-bay_pred-15min_dps-70_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.5348978963771127
RMSE: 3.195118764869727
WMAPE: 0.024576692455792748
Total sudden change in speed rate: 0.5642692815385055
WMAPE (%): 2.457669245579275
Total sudden change in speed rate (%): 56.426928153850554


In [34]:
# PeMS-BAY, 70dps, 15min, trad-fl, no graph connectivity
experiment = "2025-09-08_14-41-12_pems-bay_pred-15min_dps-70_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.526686870684145
RMSE: 3.1863538093835815
WMAPE: 0.024445691757047527
Total sudden change in speed rate: 0.5531748452586045
WMAPE (%): 2.4445691757047525
Total sudden change in speed rate (%): 55.31748452586045


In [35]:
# PeMS-BAY, 70dps, 15min, trad-fl, with algorithm
experiment = "2025-09-10_14-42-09_pems-bay_pred-15min_dps-70_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.5457374630053025
RMSE: 3.2144369186144153
WMAPE: 0.024750595229683838
Total sudden change in speed rate: 0.5601462094731816
WMAPE (%): 2.4750595229683836
Total sudden change in speed rate (%): 56.014620947318164


In [36]:
# PeMS-BAY, 140dps, 15min, trad-fl, full graph connectivity
experiment = "2025-09-05_16-04-07_pems-bay_pred-15min_dps-140_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.5253554403612024
RMSE: 3.1464441483394783
WMAPE: 0.024424227829241973
Total sudden change in speed rate: 0.5661812342413391
WMAPE (%): 2.442422782924197
Total sudden change in speed rate (%): 56.61812342413391


In [37]:
# PeMS-BAY, 140dps, 15min, trad-fl, no graph connectivity
experiment = "2025-09-08_13-39-31_pems-bay_pred-15min_dps-140_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.5184270910241806
RMSE: 3.158973686621007
WMAPE: 0.024313549052030727
Total sudden change in speed rate: 0.5595616683214197
WMAPE (%): 2.4313549052030727
Total sudden change in speed rate (%): 55.956166832141975


In [38]:
# PeMS-BAY, 140dps, 15min, trad-fl, with algorithm
experiment = "2025-09-10_09-49-06_pems-bay_pred-15min_dps-140_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.5082720799740166
RMSE: 3.135421769273173
WMAPE: 0.02415089549584766
Total sudden change in speed rate: 0.5695614974684828
WMAPE (%): 2.415089549584766
Total sudden change in speed rate (%): 56.95614974684828


In [ ]:
# PeMS-BAY, 70dps, 30min, trad-fl, full graph connectivity
experiment = "2025-09-05_22-03-31_pems-bay_pred-30min_dps-70_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.100207977977543
RMSE: 4.435748808847504
WMAPE: 0.03363306004326071
Total sudden change in speed rate: 0.45674864078364213
WMAPE (%): 3.3633060043260707
Total sudden change in speed rate (%): 45.674864078364216


In [ ]:
# PeMS-BAY, 70dps, 30min, trad-fl, no graph connectivity
experiment = "2025-09-08_15-02-02_pems-bay_pred-30min_dps-70_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.084586794700534
RMSE: 4.487622175636225
WMAPE: 0.03338425758170839
Total sudden change in speed rate: 0.43438621547550627
WMAPE (%): 3.338425758170839
Total sudden change in speed rate (%): 43.438621547550625


In [43]:
# PeMS-BAY, 70dps, 30min, trad-fl, with algorithm
experiment = "2025-09-10_16-25-54_pems-bay_pred-30min_dps-70_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.130539773423187
RMSE: 4.471296231556511
WMAPE: 0.03411832611994942
Total sudden change in speed rate: 0.4476060065548496
WMAPE (%): 3.4118326119949423
Total sudden change in speed rate (%): 44.760600655484964


In [44]:
# PeMS-BAY, 140dps, 30min, trad-fl, full graph conn.
experiment = "2025-09-05_17-33-57_pems-bay_pred-30min_dps-140_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.0316898449389535
RMSE: 4.361224504461751
WMAPE: 0.03253530653543107
Total sudden change in speed rate: 0.4647278782668891
WMAPE (%): 3.2535306535431068
Total sudden change in speed rate (%): 46.47278782668891


In [45]:
# PeMS-BAY, 140dps, 30min, trad-fl, no graph conn.
experiment = "2025-09-08_14-00-05_pems-bay_pred-30min_dps-140_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.043971346974633
RMSE: 4.4255687246904944
WMAPE: 0.03273469385026981
Total sudden change in speed rate: 0.43856727277663055
WMAPE (%): 3.273469385026981
Total sudden change in speed rate (%): 43.856727277663055


In [46]:
# PeMS-BAY, 140dps, 30min, trad-fl, with algorithm
experiment = "2025-09-10_11-23-27_pems-bay_pred-30min_dps-140_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.038461667985831
RMSE: 4.372836512233273
WMAPE: 0.03264434826271481
Total sudden change in speed rate: 0.4652427080015315
WMAPE (%): 3.264434826271481
Total sudden change in speed rate (%): 46.52427080015315


In [47]:
# PeMS-BAY, 70dps, 60min, trad-fl, full graph conn.
experiment = "2025-09-05_23-33-15_pems-bay_pred-60min_dps-70_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.718752128083388
RMSE: 5.661534436630144
WMAPE: 0.0435494703755071
Total sudden change in speed rate: 0.33264778880836254
WMAPE (%): 4.354947037550709
Total sudden change in speed rate (%): 33.26477888083625


In [48]:
# PeMS-BAY, 70dps, 60min, trad-fl, no graph conn.
experiment = "2025-09-08_15-22-41_pems-bay_pred-60min_dps-70_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.8436270823707934
RMSE: 5.869632012787157
WMAPE: 0.045553063064311526
Total sudden change in speed rate: 0.27451619651948556
WMAPE (%): 4.555306306431152
Total sudden change in speed rate (%): 27.451619651948555


In [49]:
# PeMS-BAY, 70dps, 60min, trad-fl, with algorithm
experiment = "2025-09-10_18-11-19_pems-bay_pred-60min_dps-70_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.756093213942296
RMSE: 5.702556310360254
WMAPE: 0.04414261193388698
Total sudden change in speed rate: 0.3258224961363229
WMAPE (%): 4.414261193388698
Total sudden change in speed rate (%): 32.58224961363229


In [50]:
# PeMS-BAY, 140dps, 60min, trad-fl, full graph conn.
experiment = "2025-09-05_19-03-50_pems-bay_pred-60min_dps-140_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.7029521657168623
RMSE: 5.611720574784199
WMAPE: 0.04329391445981096
Total sudden change in speed rate: 0.3404215447058811
WMAPE (%): 4.329391445981097
Total sudden change in speed rate (%): 34.04215447058811


In [51]:
# PeMS-BAY, 140dps, 60min, trad-fl, no graph conn.
experiment = "2025-09-08_14-20-44_pems-bay_pred-60min_dps-140_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.782968574496752
RMSE: 5.770048814557373
WMAPE: 0.04458055080302361
Total sudden change in speed rate: 0.29073694447609744
WMAPE (%): 4.458055080302361
Total sudden change in speed rate (%): 29.073694447609743


In [52]:
# PeMS-BAY, 140dps, 60min, trad-fl, with algorithm
experiment = "2025-09-10_13-00-40_pems-bay_pred-60min_dps-140_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.6961502937282695
RMSE: 5.620647165944566
WMAPE: 0.04318392461937116
Total sudden change in speed rate: 0.332960221503489
WMAPE (%): 4.3183924619371155
Total sudden change in speed rate (%): 33.2960221503489


In [53]:
# PeMS-BAY, 70dps, 15min, server-free-fl, full graph conn.
experiment = "2025-09-06_09-37-24_pems-bay_pred-15min_dps-70_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.550244005762804
RMSE: 3.214566428406557
WMAPE: 0.024823065839644668
Total sudden change in speed rate: 0.5586384076552683
WMAPE (%): 2.482306583964467
Total sudden change in speed rate (%): 55.86384076552683


In [54]:
# PeMS-BAY, 70dps, 15min, server-free-fl, no graph conn.
experiment = "2025-09-08_17-47-24_pems-bay_pred-15min_dps-70_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.5470227695190877
RMSE: 3.20125921003781
WMAPE: 0.024771470131335644
Total sudden change in speed rate: 0.5540269660573315
WMAPE (%): 2.4771470131335644
Total sudden change in speed rate (%): 55.40269660573315


In [55]:
# PeMS-BAY, 70dps, 15min, server-free-fl, with algorithm
experiment = "2025-09-12_15-41-00_pems-bay_pred-15min_dps-70_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.5506996368902324
RMSE: 3.221557056622491
WMAPE: 0.02483066013506507
Total sudden change in speed rate: 0.5598461830625736
WMAPE (%): 2.483066013506507
Total sudden change in speed rate (%): 55.98461830625736


In [57]:
# PeMS-BAY, 140dps, 15min, server-free-fl, full graph conn.
experiment = "2025-09-06_05-00-38_pems-bay_pred-15min_dps-140_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.507543669881879
RMSE: 3.139740949405276
WMAPE: 0.024139106774165777
Total sudden change in speed rate: 0.568396031904034
WMAPE (%): 2.4139106774165775
Total sudden change in speed rate (%): 56.839603190403395


In [58]:
# PeMS-BAY, 140dps, 15min, server-free-fl, no graph conn.
experiment = "2025-09-08_17-10-11_pems-bay_pred-15min_dps-140_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.5134852102187244
RMSE: 3.155897555070745
WMAPE: 0.02423427385230447
Total sudden change in speed rate: 0.5618525898029354
WMAPE (%): 2.423427385230447
Total sudden change in speed rate (%): 56.18525898029354


In [59]:
# PeMS-BAY, 140dps, 15min, server-free-fl, with algorithm
experiment = "2025-09-12_09-32-23_pems-bay_pred-15min_dps-140_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 1.4985558843915752
RMSE: 3.137461327667602
WMAPE: 0.023995132530413583
Total sudden change in speed rate: 0.5688053941244252
WMAPE (%): 2.3995132530413583
Total sudden change in speed rate (%): 56.88053941244252


In [ ]:
# PeMS-BAY, 70dps, 30min, server-free-fl, full graph conn.
experiment = "2025-09-06_11-09-21_pems-bay_pred-30min_dps-70_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

size: 37
size: 27
size: 33
size: 59
size: 75
size: 51
size: 43
Weighted (final epoch):
MAE: 2.077525229332245
RMSE: 4.40265574008767
WMAPE: 0.033270304552697895
Total sudden change in speed rate: 0.44531425477232056
WMAPE (%): 3.3270304552697896
Total sudden change in speed rate (%): 44.531425477232055


In [ ]:
# PeMS-BAY, 70dps, 30min, server-free-fl, no graph conn.
experiment = "2025-09-08_18-00-04_pems-bay_pred-30min_dps-70_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.0869743758542736
RMSE: 4.480987632877541
WMAPE: 0.03342213228171175
Total sudden change in speed rate: 0.4373291600584701
WMAPE (%): 3.342213228171175
Total sudden change in speed rate (%): 43.73291600584701


In [ ]:
# PeMS-BAY, 70dps, 30min, server-free-fl, with algorithm
experiment = "2025-09-12_18-14-46_pems-bay_pred-30min_dps-70_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.07805333965393
RMSE: 4.427362428290781
WMAPE: 0.03327918879685752
Total sudden change in speed rate: 0.45239960302288157
WMAPE (%): 3.3279188796857517
Total sudden change in speed rate (%): 45.239960302288154


In [ ]:
# PeMS-BAY, 70dps, 30min, server-free-fl, full graph conn.
experiment = "2025-09-06_06-32-46_pems-bay_pred-30min_dps-140_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.0420136017741384
RMSE: 4.363309776693282
WMAPE: 0.032701490885428834
Total sudden change in speed rate: 0.4679595611660761
WMAPE (%): 3.270149088542883
Total sudden change in speed rate (%): 46.79595611660761


In [ ]:
# PeMS-BAY, 70dps, 30min, server-free-fl, no graph conn.
experiment = "2025-09-08_17-22-34_pems-bay_pred-30min_dps-140_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.0478539797419413
RMSE: 4.40651221275886
WMAPE: 0.03279704151705714
Total sudden change in speed rate: 0.4512171750318266
WMAPE (%): 3.279704151705714
Total sudden change in speed rate (%): 45.121717503182666


In [ ]:
# PeMS-BAY, 70dps, 30min, server-free-fl, with algorithm
experiment = "2025-09-12_11-30-29_pems-bay_pred-30min_dps-140_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.040604055899391
RMSE: 4.372461662307756
WMAPE: 0.03268019548701059
Total sudden change in speed rate: 0.4587813792922599
WMAPE (%): 3.2680195487010586
Total sudden change in speed rate (%): 45.87813792922599


In [71]:
# PeMS-BAY, 70dps, 60min, server-free-fl, full graph conn.
experiment = "2025-09-06_12-41-40_pems-bay_pred-60min_dps-70_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.757016151624789
RMSE: 5.715637530574554
WMAPE: 0.04415692359047046
Total sudden change in speed rate: 0.32362085653695144
WMAPE (%): 4.415692359047045
Total sudden change in speed rate (%): 32.36208565369515


In [72]:
# PeMS-BAY, 70dps, 60min, server-free-fl, no graph conn.
experiment = "2025-09-08_18-12-45_pems-bay_pred-60min_dps-70_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.819066282399588
RMSE: 5.849693120640677
WMAPE: 0.04515822262333935
Total sudden change in speed rate: 0.29601968668402107
WMAPE (%): 4.515822262333935
Total sudden change in speed rate (%): 29.601968668402108


In [73]:
# PeMS-BAY, 70dps, 60min, server-free-fl, with algorithm
experiment = "2025-09-12_20-42-41_pems-bay_pred-60min_dps-70_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.7630900315942757
RMSE: 5.725909034636381
WMAPE: 0.04425309233875694
Total sudden change in speed rate: 0.3224355228663525
WMAPE (%): 4.425309233875694
Total sudden change in speed rate (%): 32.24355228663525


In [74]:
# PeMS-BAY, 140dps, 60min, server-free-fl, full graph conn.
experiment = "2025-09-06_08-05-08_pems-bay_pred-60min_dps-140_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.7463933967286365
RMSE: 5.598503781260983
WMAPE: 0.04399274046974218
Total sudden change in speed rate: 0.3411389970685105
WMAPE (%): 4.3992740469742175
Total sudden change in speed rate (%): 34.11389970685105


In [75]:
# PeMS-BAY, 140dps, 60min, server-free-fl, no graph conn.
experiment = "2025-09-08_17-34-59_pems-bay_pred-60min_dps-140_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.7477254576796164
RMSE: 5.734015337814184
WMAPE: 0.04401693617759298
Total sudden change in speed rate: 0.2992332598156945
WMAPE (%): 4.401693617759298
Total sudden change in speed rate (%): 29.923325981569448


In [76]:
# PeMS-BAY, 140dps, 60min, server-free-fl, with algorithm
experiment = "2025-09-12_13-32-51_pems-bay_pred-60min_dps-140_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.7448761452594144
RMSE: 5.606314744508168
WMAPE: 0.04396771077062382
Total sudden change in speed rate: 0.3422075607213868
WMAPE (%): 4.3967710770623825
Total sudden change in speed rate (%): 34.220756072138684


In [77]:
# PeMS-BAY, 70dps, 15min, gossip-learning, full graph conn.
experiment = "2025-09-06_22-35-25_pems-bay_pred-15min_dps-70_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 1.5713020578734957
RMSE: 3.2228776421321297
WMAPE: 0.025158710630161733
Total sudden change in speed rate: 0.5571868846030469
WMAPE (%): 2.5158710630161734
Total sudden change in speed rate (%): 55.71868846030469


In [78]:
# PeMS-BAY, 70dps, 15min, gossip-learning, no graph conn.
experiment = "2025-09-08_19-46-03_pems-bay_pred-15min_dps-70_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 1.5528561242175247
RMSE: 3.2421316518017855
WMAPE: 0.024864905714592857
Total sudden change in speed rate: 0.5524928883138963
WMAPE (%): 2.4864905714592855
Total sudden change in speed rate (%): 55.249288831389634


In [79]:
# PeMS-BAY, 70dps, 15min, gossip-learning, with algorithm
experiment = "2025-09-18_17-50-44_pems-bay_pred-15min_dps-70_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 1.7004770772737507
RMSE: 3.453118505995954
WMAPE: 0.02722977863851501
Total sudden change in speed rate: 0.5427162037558356
WMAPE (%): 2.722977863851501
Total sudden change in speed rate (%): 54.27162037558356


In [ ]:
# PeMS-BAY, 70dps, 15min, gossip-learning, with algorithm
experiment = "2025-09-18_17-50-44_pems-bay_pred-15min_dps-70_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

In [80]:
# PeMS-BAY, 140dps, 15min, gossip-learning, full graph conn.
experiment = "2025-09-06_17-59-17_pems-bay_pred-15min_dps-140_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 1.5541912249075855
RMSE: 3.2320626505257484
WMAPE: 0.024886183860277446
Total sudden change in speed rate: 0.5569956523721001
WMAPE (%): 2.4886183860277447
Total sudden change in speed rate (%): 55.69956523721002


In [81]:
# PeMS-BAY, 140dps, 15min, gossip-learning, no graph conn.
experiment = "2025-09-08_19-08-38_pems-bay_pred-15min_dps-140_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 1.5377362303446815
RMSE: 3.207311642944164
WMAPE: 0.0246230386306281
Total sudden change in speed rate: 0.555201776376948
WMAPE (%): 2.46230386306281
Total sudden change in speed rate (%): 55.5201776376948


In [82]:
# PeMS-BAY, 140dps, 15min, gossip-learning, with algorithm
experiment = "2025-09-18_12-03-51_pems-bay_pred-15min_dps-140_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 1.5780308437431991
RMSE: 3.2821240751903287
WMAPE: 0.025269184023543046
Total sudden change in speed rate: 0.5573402091945072
WMAPE (%): 2.5269184023543048
Total sudden change in speed rate (%): 55.73402091945072


In [83]:
# PeMS-BAY, 70dps, 30min, gossip-learning, full graph conn.
experiment = "2025-09-07_00-07-14_pems-bay_pred-30min_dps-70_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.126171362440789
RMSE: 4.505043430339844
WMAPE: 0.034049963490586
Total sudden change in speed rate: 0.44048294954720213
WMAPE (%): 3.4049963490586
Total sudden change in speed rate (%): 44.04829495472021


In [84]:
# PeMS-BAY, 70dps, 30min, gossip-learning, no graph conn.
experiment = "2025-09-08_19-58-48_pems-bay_pred-30min_dps-70_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.088064663359477
RMSE: 4.509200446997031
WMAPE: 0.033439410517729806
Total sudden change in speed rate: 0.42903542528514294
WMAPE (%): 3.3439410517729806
Total sudden change in speed rate (%): 42.90354252851429


In [85]:
# PeMS-BAY, 70dps, 30min, gossip-learning, with algorithm
experiment = "2025-09-18_20-15-19_pems-bay_pred-30min_dps-70_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.1616355889217824
RMSE: 4.621032816506676
WMAPE: 0.034615296122956406
Total sudden change in speed rate: 0.4290273831300004
WMAPE (%): 3.4615296122956405
Total sudden change in speed rate (%): 42.902738313000036


In [86]:
# PeMS-BAY, 140dps, 30min, gossip-learning, full graph conn.
experiment = "2025-09-06_19-31-40_pems-bay_pred-30min_dps-140_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.0991039659177706
RMSE: 4.431923731437886
WMAPE: 0.03361594922013912
Total sudden change in speed rate: 0.4429691186042309
WMAPE (%): 3.361594922013912
Total sudden change in speed rate (%): 44.29691186042309


In [87]:
# PeMS-BAY, 140dps, 30min, gossip-learning, no graph conn.
experiment = "2025-09-08_19-21-08_pems-bay_pred-30min_dps-140_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.0626482120319745
RMSE: 4.465500548129032
WMAPE: 0.03303358434815329
Total sudden change in speed rate: 0.43726103134309685
WMAPE (%): 3.303358434815329
Total sudden change in speed rate (%): 43.72610313430968


In [88]:
# PeMS-BAY, 140dps, 30min, gossip-learning, with algorithm
experiment = "2025-09-18_13-51-54_pems-bay_pred-30min_dps-140_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.0798450810330875
RMSE: 4.431863910373461
WMAPE: 0.033308479538736294
Total sudden change in speed rate: 0.44954742995688346
WMAPE (%): 3.3308479538736293
Total sudden change in speed rate (%): 44.95474299568835


In [89]:
# PeMS-BAY, 70dps, 60min, gossip-learning, full graph conn.
experiment = "2025-09-07_01-39-17_pems-bay_pred-60min_dps-70_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.790367450296063
RMSE: 5.731221985303816
WMAPE: 0.0446907439016196
Total sudden change in speed rate: 0.31577385738894564
WMAPE (%): 4.4690743901619605
Total sudden change in speed rate (%): 31.577385738894563


In [90]:
# PeMS-BAY, 70dps, 60min, gossip-learning, no graph conn.
experiment = "2025-09-08_20-11-31_pems-bay_pred-60min_dps-70_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.850422574942812
RMSE: 5.903217374346772
WMAPE: 0.04565822939037299
Total sudden change in speed rate: 0.2696398432999362
WMAPE (%): 4.565822939037299
Total sudden change in speed rate (%): 26.963984329993618


In [91]:
# PeMS-BAY, 70dps, 60min, gossip-learning, with algorithm
experiment = "2025-09-18_22-37-44_pems-bay_pred-60min_dps-70_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.9097690850870572
RMSE: 5.982876822470185
WMAPE: 0.046605529440294606
Total sudden change in speed rate: 0.30980496565562843
WMAPE (%): 4.660552944029461
Total sudden change in speed rate (%): 30.98049656556284


In [92]:
# PeMS-BAY, 140dps, 60min, gossip-learning, full graph conn.
experiment = "2025-09-06_21-03-28_pems-bay_pred-60min_dps-140_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.7531460439505686
RMSE: 5.635469755568912
WMAPE: 0.04409932056755879
Total sudden change in speed rate: 0.3283902148592899
WMAPE (%): 4.409932056755879
Total sudden change in speed rate (%): 32.83902148592899


In [93]:
# PeMS-BAY, 140dps, 60min, gossip-learning, no graph conn.
experiment = "2025-09-08_19-33-37_pems-bay_pred-60min_dps-140_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.8086202810133507
RMSE: 5.865850331559154
WMAPE: 0.04499243056097424
Total sudden change in speed rate: 0.2810570647637178
WMAPE (%): 4.4992430560974235
Total sudden change in speed rate (%): 28.10570647637178


In [95]:
# PeMS-BAY, 140dps, 60min, gossip-learning, with algorithm
experiment = "2025-09-18_15-45-43_pems-bay_pred-60min_dps-140_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.8329514447450896
RMSE: 5.8591006034278035
WMAPE: 0.045377539330734455
Total sudden change in speed rate: 0.28317106411749515
WMAPE (%): 4.5377539330734455
Total sudden change in speed rate (%): 28.317106411749513


In [6]:
cln_nodes_list = get_num_nodes_per_cloudlet("pemsd7-m", "experiment_1")
cloudlet_sizes = {i: len(nodes) for i, nodes in enumerate(cln_nodes_list)}

In [98]:
# PeMSD7-M, 70dps, 15min, trad-fl, full graph conn.
experiment = "2025-09-06_03-01-52_pemsd7-m_pred-15min_dps-70_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.5068527151728093
RMSE: 4.476267702428891
WMAPE: 0.04341497179351308
Total sudden change in speed rate: 0.5475161527934805
WMAPE (%): 4.3414971793513075
Total sudden change in speed rate (%): 54.75161527934805


In [99]:
# PeMSD7-M, 70dps, 15min, trad-fl, no graph conn.
experiment = "2025-09-08_16-26-30_pemsd7-m_pred-15min_dps-70_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.5591352243772767
RMSE: 4.538576853190434
WMAPE: 0.0443244584420958
Total sudden change in speed rate: 0.5331347025802697
WMAPE (%): 4.43244584420958
Total sudden change in speed rate (%): 53.313470258026975


In [100]:
# PeMSD7-M, 70dps, 15min, trad-fl, with algorithm
experiment = "2025-09-10_21-53-05_pemsd7-m_pred-15min_dps-70_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.592116497600816
RMSE: 4.596663465836262
WMAPE: 0.04488448466724105
Total sudden change in speed rate: 0.5405319284528427
WMAPE (%): 4.488448466724105
Total sudden change in speed rate (%): 54.05319284528427


In [101]:
# PeMSD7-M, 140dps, 15min, trad-fl, full graph conn.
experiment = "2025-09-06_01-03-18_pemsd7-m_pred-15min_dps-140_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.461974816647684
RMSE: 4.374761958759442
WMAPE: 0.04263391920314736
Total sudden change in speed rate: 0.55496796998036
WMAPE (%): 4.263391920314736
Total sudden change in speed rate (%): 55.496796998036004


In [102]:
# PeMSD7-M, 140dps, 15min, trad-fl, no graph conn.
experiment = "2025-09-08_15-43-26_pemsd7-m_pred-15min_dps-140_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.44745588954937
RMSE: 4.416743752253923
WMAPE: 0.0423876828913029
Total sudden change in speed rate: 0.5387806142154522
WMAPE (%): 4.23876828913029
Total sudden change in speed rate (%): 53.87806142154522


In [103]:
# PeMSD7-M, 140dps, 15min, trad-fl, with algorithm
experiment = "2025-09-10_20-02-01_pemsd7-m_pred-15min_dps-140_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.440094271342511
RMSE: 4.393785084112668
WMAPE: 0.042255247375121076
Total sudden change in speed rate: 0.5477694266938672
WMAPE (%): 4.225524737512107
Total sudden change in speed rate (%): 54.77694266938672


In [104]:
# PeMSD7-M, 70dps, 30min, trad-fl, full graph conn.
experiment = "2025-09-06_03-41-41_pemsd7-m_pred-30min_dps-70_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4243927932688636
RMSE: 6.185069446596777
WMAPE: 0.05931864780343182
Total sudden change in speed rate: 0.4182008746746924
WMAPE (%): 5.931864780343182
Total sudden change in speed rate (%): 41.82008746746924


In [105]:
# PeMSD7-M, 70dps, 30min, trad-fl, no graph conn.
experiment = "2025-09-08_16-40-57_pemsd7-m_pred-30min_dps-70_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4961599739514626
RMSE: 6.348899830982957
WMAPE: 0.0605749342144733
Total sudden change in speed rate: 0.3787817669746523
WMAPE (%): 6.0574934214473295
Total sudden change in speed rate (%): 37.878176697465236


In [106]:
# PeMSD7-M, 70dps, 30min, trad-fl, with algorithm
experiment = "2025-09-10_22-31-57_pemsd7-m_pred-30min_dps-70_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4464283042775907
RMSE: 6.195685753139758
WMAPE: 0.05969894652212261
Total sudden change in speed rate: 0.41678130099552513
WMAPE (%): 5.969894652212261
Total sudden change in speed rate (%): 41.67813009955251


In [107]:
# PeMSD7-M, 140dps, 30min, trad-fl, full graph conn.
experiment = "2025-09-06_01-42-41_pemsd7-m_pred-30min_dps-140_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.395613559277637
RMSE: 6.086122922186373
WMAPE: 0.05884849289849185
Total sudden change in speed rate: 0.43080657092408126
WMAPE (%): 5.8848492898491855
Total sudden change in speed rate (%): 43.080657092408124


In [108]:
# PeMSD7-M, 140dps, 30min, trad-fl, no graph conn.
experiment = "2025-09-08_15-57-43_pemsd7-m_pred-30min_dps-140_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4150137786994508
RMSE: 6.1740004290103006
WMAPE: 0.05917571196146033
Total sudden change in speed rate: 0.3955297150472465
WMAPE (%): 5.917571196146032
Total sudden change in speed rate (%): 39.552971504724646


In [109]:
# PeMSD7-M, 140dps, 30min, trad-fl, with algorithm
experiment = "2025-09-10_20-38-50_pemsd7-m_pred-30min_dps-140_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.371577442159507
RMSE: 6.100157325199781
WMAPE: 0.058417672831041614
Total sudden change in speed rate: 0.42807868749764727
WMAPE (%): 5.841767283104161
Total sudden change in speed rate (%): 42.80786874976473


In [110]:
# PeMSD7-M, 70dps, 60min, trad-fl, full graph conn.
experiment = "2025-09-06_04-21-11_pemsd7-m_pred-60min_dps-70_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.6721519337107384
RMSE: 8.104060445716506
WMAPE: 0.08093547499742584
Total sudden change in speed rate: 0.2880189428884179
WMAPE (%): 8.093547499742584
Total sudden change in speed rate (%): 28.80189428884179


In [111]:
# PeMSD7-M, 70dps, 60min, trad-fl, no graph conn.
experiment = "2025-09-08_16-55-33_pemsd7-m_pred-60min_dps-70_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.789292659574212
RMSE: 8.279715905401646
WMAPE: 0.08299399707197203
Total sudden change in speed rate: 0.2424735154304263
WMAPE (%): 8.299399707197203
Total sudden change in speed rate (%): 24.24735154304263


In [112]:
# PeMSD7-M, 70dps, 60min, trad-fl, with algorithm
experiment = "2025-09-10_23-09-27_pemsd7-m_pred-60min_dps-70_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.695178791169659
RMSE: 8.198383239729601
WMAPE: 0.08137709794687138
Total sudden change in speed rate: 0.3111131197457055
WMAPE (%): 8.137709794687138
Total sudden change in speed rate (%): 31.111311974570548


In [113]:
# PeMSD7-M, 140dps, 60min, trad-fl, full graph conn.
experiment = "2025-09-06_02-22-13_pemsd7-m_pred-60min_dps-140_trad-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.56139904880387
RMSE: 7.811518475980011
WMAPE: 0.07906226342839873
Total sudden change in speed rate: 0.30842515609777993
WMAPE (%): 7.906226342839872
Total sudden change in speed rate (%): 30.842515609777994


In [114]:
# PeMSD7-M, 140dps, 60min, trad-fl, no graph conn.
experiment = "2025-09-08_16-12-09_pemsd7-m_pred-60min_dps-140_trad-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.651818066907477
RMSE: 8.090978990335879
WMAPE: 0.08064231014058937
Total sudden change in speed rate: 0.2583767014217754
WMAPE (%): 8.064231014058937
Total sudden change in speed rate (%): 25.837670142177537


In [115]:
# PeMSD7-M, 140dps, 60min, trad-fl, with algorithm
experiment = "2025-09-10_21-15-40_pemsd7-m_pred-60min_dps-140_trad-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.627282563757829
RMSE: 7.9504226640969415
WMAPE: 0.08021581857150781
Total sudden change in speed rate: 0.29093934232809904
WMAPE (%): 8.02158185715078
Total sudden change in speed rate (%): 29.093934232809904


In [ ]:
# PeMSD7-M, 70dps, 15min, server-free-fl, full graph conn.
experiment = "2025-09-06_16-06-16_pemsd7-m_pred-15min_dps-70_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.5893406984099236
RMSE: 4.6187344489517725
WMAPE: 0.044845007736807156
Total sudden change in speed rate: 0.5410133498001163
WMAPE (%): 4.484500773680716
Total sudden change in speed rate (%): 54.10133498001163


In [ ]:
# PeMSD7-M, 70dps, 15min, server-free-fl, no graph conn.
experiment = "2025-09-08_18-46-43_pemsd7-m_pred-15min_dps-70_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.5607986089096184
RMSE: 4.633612835026279
WMAPE: 0.04437041314366039
Total sudden change in speed rate: 0.5344863949000678
WMAPE (%): 4.437041314366039
Total sudden change in speed rate (%): 53.44863949000678


In [9]:
# PeMSD7-M, 70dps, 15min, server-free-fl, with algorithm
experiment = "2025-09-13_01-14-49_pemsd7-m_pred-15min_dps-70_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.5599262074788958
RMSE: 4.585005333838443
WMAPE: 0.0443407543621093
Total sudden change in speed rate: 0.5408770670785779
WMAPE (%): 4.43407543621093
Total sudden change in speed rate (%): 54.087706707857784


In [10]:
# PeMSD7-M, 140dps, 15min, server-free-fl, full graph conn.
experiment = "2025-09-06_14-13-47_pemsd7-m_pred-15min_dps-140_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.4445623864752823
RMSE: 4.376943334924727
WMAPE: 0.04233527922870678
Total sudden change in speed rate: 0.5570642836069652
WMAPE (%): 4.233527922870678
Total sudden change in speed rate (%): 55.70642836069653


In [11]:
# PeMSD7-M, 140dps, 15min, server-free-fl, no graph conn.
experiment = "2025-09-08_18-25-25_pemsd7-m_pred-15min_dps-140_server-free-fl-distance_online-training_no_graph-connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.462194696015732
RMSE: 4.420169390734111
WMAPE: 0.04263107265779694
Total sudden change in speed rate: 0.5436616678008113
WMAPE (%): 4.263107265779694
Total sudden change in speed rate (%): 54.36616678008113


In [ ]:
# PeMSD7-M, 140dps, 15min, server-free-fl, with algorithm
experiment = "2025-09-12_23-28-09_pemsd7-m_pred-15min_dps-140_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.443621987123636
RMSE: 4.388113152045629
WMAPE: 0.042320393056591354
Total sudden change in speed rate: 0.5513774881489135
WMAPE (%): 4.232039305659136
Total sudden change in speed rate (%): 55.13774881489135


In [13]:
# PeMSD7-M, 70dps, 30min, server-free-fl, full graph conn.
experiment = "2025-09-06_16-43-58_pemsd7-m_pred-30min_dps-70_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.496694361227904
RMSE: 6.254676684099559
WMAPE: 0.06055748599851275
Total sudden change in speed rate: 0.4135629745359036
WMAPE (%): 6.055748599851276
Total sudden change in speed rate (%): 41.356297453590365


In [14]:
# PeMSD7-M, 70dps, 30min, server-free-fl, no graph conn.
experiment = "2025-09-08_18-54-02_pemsd7-m_pred-30min_dps-70_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4538163157236625
RMSE: 6.323683811825496
WMAPE: 0.059834051501691285
Total sudden change in speed rate: 0.38265730632546874
WMAPE (%): 5.983405150169128
Total sudden change in speed rate (%): 38.26573063254688


In [15]:
# PeMSD7-M, 70dps, 30min, server-free-fl, with algorithm
experiment = "2025-09-13_01-53-59_pemsd7-m_pred-30min_dps-70_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4548156430259915
RMSE: 6.194888399860672
WMAPE: 0.05984616581352676
Total sudden change in speed rate: 0.4232065993830797
WMAPE (%): 5.984616581352676
Total sudden change in speed rate (%): 42.32065993830797


In [16]:
# PeMSD7-M, 140dps, 30min, server-free-fl, full graph conn.
experiment = "2025-09-06_14-51-20_pemsd7-m_pred-30min_dps-140_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.3677153507783664
RMSE: 6.054273517727853
WMAPE: 0.05835285426295092
Total sudden change in speed rate: 0.42886542264611976
WMAPE (%): 5.835285426295092
Total sudden change in speed rate (%): 42.886542264611975


In [17]:
# PeMSD7-M, 140dps, 30min, server-free-fl, no graph conn.
experiment = "2025-09-08_18-32-31_pemsd7-m_pred-30min_dps-140_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.3838990546160916
RMSE: 6.147927335164401
WMAPE: 0.05863458833642754
Total sudden change in speed rate: 0.41577550667876007
WMAPE (%): 5.863458833642754
Total sudden change in speed rate (%): 41.577550667876004


In [18]:
# PeMSD7-M, 140dps, 30min, server-free-fl, with algorithm
experiment = "2025-09-13_00-03-24_pemsd7-m_pred-30min_dps-140_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.389543788794929
RMSE: 6.0612132601765065
WMAPE: 0.058726849791700377
Total sudden change in speed rate: 0.4357133544614194
WMAPE (%): 5.872684979170038
Total sudden change in speed rate (%): 43.57133544614194


In [ ]:
# PeMSD7-M, 70dps, 60min, server-free-fl, full graph conn.
experiment = "2025-09-06_17-21-37_pemsd7-m_pred-60min_dps-70_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.5843876922056435
RMSE: 8.09921018028386
WMAPE: 0.07945558818467191
Total sudden change in speed rate: 0.31127728390993
WMAPE (%): 7.945558818467191
Total sudden change in speed rate (%): 31.127728390993


In [20]:
# PeMSD7-M, 70dps, 60min, server-free-fl, no graph conn.
experiment = "2025-09-08_19-01-20_pemsd7-m_pred-60min_dps-70_server-free-fl-distance_online-training_no_graph-connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.786121845746128
RMSE: 8.339024543099853
WMAPE: 0.08295122812214901
Total sudden change in speed rate: 0.2409144174565436
WMAPE (%): 8.2951228122149
Total sudden change in speed rate (%): 24.09144174565436


In [21]:
# PeMSD7-M, 70dps, 60min, server-free-fl, with algorithm
experiment = "2025-09-13_02-33-58_pemsd7-m_pred-60min_dps-70_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.741853666083071
RMSE: 8.119228239196273
WMAPE: 0.08216097796578252
Total sudden change in speed rate: 0.3037202532295462
WMAPE (%): 8.216097796578252
Total sudden change in speed rate (%): 30.37202532295462


In [22]:
# PeMSD7-M, 140dps, 60min, server-free-fl, full graph conn.
experiment = "2025-09-06_15-28-48_pemsd7-m_pred-60min_dps-140_server-free-fl-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.537769797178753
RMSE: 7.808900328614415
WMAPE: 0.0786588726366468
Total sudden change in speed rate: 0.31649012107734487
WMAPE (%): 7.865887263664679
Total sudden change in speed rate (%): 31.649012107734485


In [23]:
# PeMSD7-M, 140dps, 60min, server-free-fl, no graph conn.
experiment = "2025-09-08_18-39-37_pemsd7-m_pred-60min_dps-140_server-free-fl-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.64246381840683
RMSE: 8.09114597425665
WMAPE: 0.08048910481913338
Total sudden change in speed rate: 0.2656235919210134
WMAPE (%): 8.048910481913337
Total sudden change in speed rate (%): 26.562359192101344


In [24]:
# PeMSD7-M, 140dps, 60min, server-free-fl, with algorithm
experiment = "2025-09-13_00-39-09_pemsd7-m_pred-60min_dps-140_server-free-fl-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.5918449800586245
RMSE: 7.903825288862433
WMAPE: 0.0795925007895249
Total sudden change in speed rate: 0.30162411660818367
WMAPE (%): 7.95925007895249
Total sudden change in speed rate (%): 30.162411660818368


In [25]:
# PeMSD7-M, 70dps, 15min, gossip-learning, full graph conn.
experiment = "2025-09-07_05-04-00_pemsd7-m_pred-15min_dps-70_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.702928004632764
RMSE: 4.764769924488276
WMAPE: 0.04681011845308093
Total sudden change in speed rate: 0.5310699158487603
WMAPE (%): 4.6810118453080936
Total sudden change in speed rate (%): 53.10699158487603


In [26]:
# PeMSD7-M, 70dps, 15min, gossip-learning, no graph conn.
experiment = "2025-09-08_20-45-34_pemsd7-m_pred-15min_dps-70_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.774491248045416
RMSE: 4.888755438055374
WMAPE: 0.04808629735027304
Total sudden change in speed rate: 0.5166087750013946
WMAPE (%): 4.808629735027304
Total sudden change in speed rate (%): 51.660877500139456


In [27]:
# PeMSD7-M, 70dps, 15min, gossip-learning, with algorithm
experiment = "2025-09-19_03-04-28_pemsd7-m_pred-15min_dps-70_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.890728296237983
RMSE: 5.005921536901969
WMAPE: 0.050093423342689215
Total sudden change in speed rate: 0.5185653354670406
WMAPE (%): 5.009342334268921
Total sudden change in speed rate (%): 51.856533546704064


In [28]:
# PeMSD7-M, 140dps, 15min, gossip-learning, full graph conn.
experiment = "2025-09-07_03-11-24_pemsd7-m_pred-15min_dps-140_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.5028872338234853
RMSE: 4.438947256602224
WMAPE: 0.04334196036981541
Total sudden change in speed rate: 0.5518872860004435
WMAPE (%): 4.334196036981542
Total sudden change in speed rate (%): 55.18872860004434


In [29]:
# PeMSD7-M, 140dps, 15min, gossip-learning, no graph conn.
experiment = "2025-09-08_20-24-17_pemsd7-m_pred-15min_dps-140_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.5611189987553797
RMSE: 4.547508135581484
WMAPE: 0.044360820678289385
Total sudden change in speed rate: 0.5374306080277013
WMAPE (%): 4.436082067828939
Total sudden change in speed rate (%): 53.743060802770124


In [30]:
# PeMSD7-M, 140dps, 15min, gossip-learning, with algorithm
experiment = "2025-09-19_01-18-27_pemsd7-m_pred-15min_dps-140_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 2.5644786792306147
RMSE: 4.555893983891465
WMAPE: 0.044420303583469156
Total sudden change in speed rate: 0.5426104564152147
WMAPE (%): 4.442030358346916
Total sudden change in speed rate (%): 54.26104564152146


In [31]:
# PeMSD7-M, 70dps, 30min, gossip-learning, full graph conn.
experiment = "2025-09-07_05-41-42_pemsd7-m_pred-30min_dps-70_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.741725442760497
RMSE: 6.629562628345978
WMAPE: 0.06482756432124871
Total sudden change in speed rate: 0.3807265724168551
WMAPE (%): 6.482756432124871
Total sudden change in speed rate (%): 38.07265724168551


In [32]:
# PeMSD7-M, 70dps, 30min, gossip-learning, no graph conn.
experiment = "2025-09-08_20-52-54_pemsd7-m_pred-30min_dps-70_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.5785792119595516
RMSE: 6.4619068893402165
WMAPE: 0.06201179952216743
Total sudden change in speed rate: 0.3640998636756382
WMAPE (%): 6.201179952216743
Total sudden change in speed rate (%): 36.40998636756382


In [33]:
# PeMSD7-M, 70dps, 30min, gossip-learning, with algorithm
experiment = "2025-09-19_03-42-05_pemsd7-m_pred-30min_dps-70_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.651358694997694
RMSE: 6.399380208386124
WMAPE: 0.06325378091751065
Total sudden change in speed rate: 0.41782953555756713
WMAPE (%): 6.325378091751064
Total sudden change in speed rate (%): 41.78295355575671


In [34]:
# PeMSD7-M, 140dps, 30min, gossip-learning, full graph conn.
experiment = "2025-09-07_03-48-56_pemsd7-m_pred-30min_dps-140_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4836247268530847
RMSE: 6.165707748253391
WMAPE: 0.060364560144489404
Total sudden change in speed rate: 0.4156353210099349
WMAPE (%): 6.03645601444894
Total sudden change in speed rate (%): 41.56353210099349


In [35]:
# PeMSD7-M, 140dps, 30min, gossip-learning, no graph conn.
experiment = "2025-09-08_20-31-23_pemsd7-m_pred-30min_dps-140_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4964954534494788
RMSE: 6.325148054572633
WMAPE: 0.06060655599322734
Total sudden change in speed rate: 0.38707255488869613
WMAPE (%): 6.060655599322734
Total sudden change in speed rate (%): 38.70725548886961


In [ ]:
# PeMSD7-M, 140dps, 30min, gossip-learning, with algorithm
experiment = "2025-09-19_01-52-17_pemsd7-m_pred-30min_dps-140_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 3.4444375073831957
RMSE: 6.16159884184197
WMAPE: 0.059691845310688385
Total sudden change in speed rate: 0.41495386167601755
WMAPE (%): 5.969184531068839
Total sudden change in speed rate (%): 41.49538616760176


In [37]:
# PeMSD7-M, 70dps, 60min, gossip-learning, full graph conn.
experiment = "2025-09-07_06-19-22_pemsd7-m_pred-60min_dps-70_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.732050109490364
RMSE: 8.175959039684754
WMAPE: 0.0819607550785501
Total sudden change in speed rate: 0.2925047126889639
WMAPE (%): 8.19607550785501
Total sudden change in speed rate (%): 29.250471268896387


In [38]:
# PeMSD7-M, 70dps, 60min, gossip-learning, no graph conn.
experiment = "2025-09-08_21-00-14_pemsd7-m_pred-60min_dps-70_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.92605601249938
RMSE: 8.4175603345295
WMAPE: 0.08539593843320797
Total sudden change in speed rate: 0.24343428202252185
WMAPE (%): 8.539593843320798
Total sudden change in speed rate (%): 24.343428202252184


In [39]:
# PeMSD7-M, 70dps, 60min, gossip-learning, with algorithm
experiment = "2025-09-19_04-18-57_pemsd7-m_pred-60min_dps-70_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.928856015022074
RMSE: 8.401499441978208
WMAPE: 0.08540903575644744
Total sudden change in speed rate: 0.29170594977045794
WMAPE (%): 8.540903575644744
Total sudden change in speed rate (%): 29.170594977045795


In [40]:
# PeMSD7-M, 140dps, 60min, gossip-learning, full graph conn.
experiment = "2025-09-07_04-26-26_pemsd7-m_pred-60min_dps-140_gossip-learning-distance_online-training_no_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.607975422887755
RMSE: 8.007276543983682
WMAPE: 0.07987668869811858
Total sudden change in speed rate: 0.3244514295046027
WMAPE (%): 7.987668869811857
Total sudden change in speed rate (%): 32.44514295046027


In [41]:
# PeMSD7-M, 140dps, 60min, gossip-learning, no graph conn.
experiment = "2025-09-08_20-38-29_pemsd7-m_pred-60min_dps-140_gossip-learning-distance_online-training_no_graph_connectivity"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.763155420724556
RMSE: 8.188078410140491
WMAPE: 0.08256001241769023
Total sudden change in speed rate: 0.265140982775101
WMAPE (%): 8.256001241769024
Total sudden change in speed rate (%): 26.5140982775101


In [42]:
# PeMSD7-M, 140dps, 60min, gossip-learning, with algorithm
experiment = "2025-09-19_02-27-38_pemsd7-m_pred-60min_dps-140_gossip-learning-distance_online-training_with_algorithm"
res = compute_weighted_final_metrics(experiment, cloudlet_sizes)
print("Weighted (final epoch):")
for k, v in res.items():
    print(f"{k}: {v}")
# If you want WMAPE and sudden rate printed as percentages:
print(f"WMAPE (%): {res['WMAPE']*100}")
print(f"Total sudden change in speed rate (%): {res['Total sudden change in speed rate']*100}")

Weighted (final epoch):
MAE: 4.733884950480539
RMSE: 8.085895744233262
WMAPE: 0.08205922250722521
Total sudden change in speed rate: 0.30231300983932297
WMAPE (%): 8.205922250722521
Total sudden change in speed rate (%): 30.231300983932297
